# Minimax Algorithm with alpha beta prunning

This code is a mix between the Kaggle competition Tutorial and what Keith Galli did in his minimax algorithm in his youtube video about creating a 4Connect app
I highly recomed his series: https://www.youtube.com/watch?v=UYgyRArKDEs&list=PLFCB5Dp81iNV_inzM-R9AKkZZlePCZdtV

The key is to play around the score you assing to each possition of the board - please refer to his video.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Code Itself

In [ ]:
# Agent Little_Pumba
# Imports


#Create a semi cleaver agent

def little_agent_pumba(obs, config):
    
    import math
    import random
    import numpy as np
    
    
    # Check the score of every board
    def board_posicion_score(board,piece,config):
        score = 0
        COL_COUNT = config.columns
        ROW_COUNT = config.rows

        # Score center
        center_array = [int(i) for i in list(board[:,COL_COUNT//2])]
        center_count = center_array.count(piece)
        score += center_count *2



        # Score Horizontal
        for r in range(ROW_COUNT):
            row_array = [int(i) for i in board[r,:]]
            for c in range(COL_COUNT-(config.inarow-1)):
                window = row_array[c:c+config.inarow]
                score += evaluate_window(window,piece)


        # Score Vertical
        for c in range(COL_COUNT):
            col_array = [int(i) for i in board[:,c]]
            for r in range(ROW_COUNT-(config.inarow-1)):
                window = col_array[r:r+config.inarow]
                score += evaluate_window(window,piece)


        # Score diagonal positive
        for r in range(ROW_COUNT-(config.inarow-1)):
            for c in range(COL_COUNT-(config.inarow-1)):
                window = [board[r+i][c+i] for i in range(config.inarow)]
                score += evaluate_window(window,piece)

        # Score diagonal negative
        for r in range((config.inarow-1),ROW_COUNT):
            for c in range(COL_COUNT-(config.inarow-1)):
                window = [board[r-i][c+i] for i in range(config.inarow)]
                score += evaluate_window(window,piece)
        return score



    # Simulate the drop of the piece

    def drop_piece(board, col, piece, config):
        next_grid = board
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid



    # Evaluate the score of each window (Horizontal, Vertical, Positive Diagonal, Negative Diagonal - Tunning this scores you can get better score)
    def evaluate_window(window,piece):
        score = 0
        EMPTY =0
        opp_piece=1
        if piece == 1:
            opp_piece =2


        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(EMPTY) ==1:
            score += 15
        elif window.count(piece) == 2 and window.count(EMPTY) == 2:
            score += 5

        if window.count(opp_piece) == 3 and window.count(EMPTY) ==1:
            score -= 80

        elif window.count(opp_piece) == 2 and window.count(EMPTY) == 2:
             score -= 20
        return score



    # Is it a winning move?
    def winning_move(board, player,config):
        # Horizontal winning
        COL_COUNT = config.columns
        ROW_COUNT = config.rows


        for c in range(COL_COUNT-(config.inarow-1)):
            for r in range(ROW_COUNT):
                if [board[r][c+i]== player for i in range(config.inarow)] == [True for j in range(config.inarow)]:

                    return True

        # Vertical winning
        for c in range(COL_COUNT):
            for r in range(ROW_COUNT-(config.inarow-1)):
                if [board[r+i][c]== player for i in range(config.inarow)] == [True for j in range(config.inarow)]:
                    return True


        # Positive Diagonal winning
        for c in range(COL_COUNT-(config.inarow-1)):
            for r in range(ROW_COUNT-(config.inarow-1)):
                if [board[r+i][c+i]== player for i in range(config.inarow)] == [True for j in range(config.inarow)]:

                    return True

        # Negative Diagonal winning
        for c in range(COL_COUNT-(config.inarow-1)):
            for r in range((config.inarow-1),ROW_COUNT):
                if [board[r-i][c+i]== player for i in range(config.inarow)] == [True for j in range(config.inarow)]:

                    return True




    def is_terminal_node(board,valid_locations,piece,opp_piece,config):
        return winning_move(board, opp_piece,config) or winning_move(board, piece,config) or len(valid_locations)==0


    # Implemetation of the algorithm
    def minimax_algorithm(board,depth,alpha,beta,maxmizingPlayer,obs,config,piece,opp_piece):
        valid_locations = [c for c in range(config.columns) if obs.board[c] == 0]


        if depth == 0 or is_terminal_node(board,valid_locations,piece,opp_piece,config):
            #print(is_terminal_node(board,valid_locations,piece,opp_piece,config))
            if is_terminal_node:
                if winning_move(board,piece,config):
                    return (None, math.inf)
                elif winning_move(board,opp_piece,config):
                    return(None,-math.inf)
                else: 
                    return (None, board_posicion_score(board,piece,config))

        if maxmizingPlayer:
            value = -math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                board_copy = board.copy()
                new_board = drop_piece(board_copy,col,piece,config)
                new_score = minimax_algorithm(new_board,depth-1,alpha,beta,False,obs,config,piece,opp_piece)[1]
                #print('AI -- Col {}, Value {}'.format(col,new_score))
                #print(new_board)
                if new_score > value:
                    value = new_score
                    best_col = col
                alpha = max(alpha,value)
                if alpha >= beta:
                    break
            return best_col, value

        else: # Minimazing player
            value = math.inf
            best_col = random.choice(valid_locations)
            for col in valid_locations:
                board_copy = board.copy()
                new_board = drop_piece(board_copy,col,opp_piece,config)
                new_score =minimax_algorithm(new_board,depth-1,alpha,beta,True,obs,config,piece,opp_piece)[1]
                #print('PLAYER -- Col {}, Value {}'.format(col,new_score))
                #print(new_board)
                if new_score < value:
                    value = new_score
                    best_col = col
                beta = min(beta,value)
                if beta <=alpha:
                    break

            return best_col, value







    # Deep level is 2 - For a reason I do not know with a deepness of 2 is working better than with deep = 3
    
    
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    #print(grid)
    piece = obs.mark
    opp_piece =1
    if piece == 1:
        opp_piece=2
    
    best_col, minmax_score = minimax_algorithm(grid,2,-math.inf,math.inf,True,obs,config,piece,opp_piece)
    
    return best_col

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(little_agent_pumba, "./submission.py")